In [112]:
"""
Creating alltogether a new graph is not a good idea
It will require a lot of deep cloning!

We need to update children, and after collapse they should happen 
"""
%run BNRep.ipynb

>> A
>> B
>> C

A 	| B,	 C 	| Prob
1 	| True,	 Haku 	| 0.1
1 	| True,	 Mata 	| 0.2
1 	| Fals,	 Haku 	| 0.3
1 	| Fals,	 Mata 	| 0.4
2 	| True,	 Haku 	| 0.5
2 	| True,	 Mata 	| 0.6
2 	| Fals,	 Haku 	| 0.7
2 	| Fals,	 Mata 	| 0.8
3 	| True,	 Haku 	| 0.9
3 	| True,	 Mata 	| 1.0
3 	| Fals,	 Haku 	| 1.1
3 	| Fals,	 Mata 	| 1.2


In [113]:
G = []
A = Vertex("A: Metastatic Cancer")
B = Vertex("B: Increased total serum calcium")
C = Vertex("C: Brain tumor")
D = Vertex("D: Coma")
E = Vertex("E: Severe headaches")

A.values = [False, True]
B.values = [False, True]
C.values = [False, True]
D.values = [False, True]
E.values = [False, True]

a_cpt = CPT([A], [0.8, 0.2])
b_cpt = CPT([B, A], [0.8, 0.2, 0.2, 0.8])
c_cpt = CPT([C, A], [0.95, 0.8, 0.05, 0.2])
d_cpt = CPT([D, B, C], [0.95, 0.2, 0.2, 0.2, 
                        0.05, 0.8, 0.8, 0.8])
e_cpt = CPT([E, C], [0.4, 0.2, 0.6, 0.8])
A.CPT = a_cpt
B.CPT = b_cpt
C.CPT = c_cpt
D.CPT = d_cpt
E.CPT = e_cpt

G = [A, B, C, D, E]

In [114]:
def setChildren(G):
    """
    Given a graph, as list of nodes, it'll update 
    children for all nodes
    """
    for n in G:
        n.children = []
        n.is_init = False
    for n in G:
        for p in n.parents:
            p.children += [n]
    for n in G:
        n.initializeV()

In [115]:
def collapse(G, nodes, newName):
    """
    It will return a new graph (aka list of nodes)
    where new graph will replace all of the nodes by newNode
    """
    setChildren(G)
    nG = [N for N in G if N not in nodes]
    pSet = set()
    cSet = set()
    for n in nodes:
        pSet = pSet.union(set(n.parents))
        cSet = cSet.union(set(n.children))
    nNode = Vertex(newName)
    nNode.parents = list(pSet)
    nNode.children = list(cSet)
    nNode.origG = G
    nNode.origNodes = nodes
    nG += [nNode]
    pCard = np.prod(#[1] +  #So that we have 2 values
            [n.nCard() for n in nNode.parents])
    #card = prod of all avlues taken by nodes
    card = np.prod([n.nCard() for n in nodes])
    nNode.values = [i for i in range(card)]
    #
    #Now, calculate newNode|it's parents
    probs = []
    nodesVals = [0]*len(nodes)
    nNodeMap = {}
    for nV in nNode.values:  #Till card
        if nV != 0: #then increment values in nodesVals
            for idx in range(len(nodes)-1, -1, -1):
                nodesVals[idx] += 1
                if nodesVals[idx] < nodes[idx].nCard():
                        break
                nodesVals[idx] = 0
        nNodeMap[nV] = [v for v in nodesVals] #Trying to copy 
        # the list At this point we have values for all the nodes
        #Now for each value we'll fill values for all the parents!
        pVals = [0]*len(nNode.parents)
        for pIdx in range(pCard):
            if pIdx != 0:
                for idx in range(len(pVals)-1, -1, -1):
                    pVals[idx] += 1
                    if pVals[idx] < nNode.parents[idx].nCard():
                        break
                    pVals[idx] = 0
            t_prob = 1
            for nIdx in range(len(nodes)):
                #Find the probability of this node, given
                # the value it takes along with it's parent
                nArgs = [nodesVals[nIdx]]
                node = nodes[nIdx]
                for p in node.parents:
                    nArgs += [pVals[nNode.parents.index(p)]]
                t_prob *= node.CPT.getP4mIdx(nArgs)
            probs += [t_prob]
    nNode.CPT = CPT([nNode]+nNode.parents, probs)
    #
    #Now, CPT of children would also need to be updated!!
    #p(c|z) = p(c|a,b) = p(c|a): meaning entries may repeat 
    # if not all nodes are parents.
    #
    #Simplest way would be to query original node for the value!!
    #z value would map to values of existing node!!
    print("Now handling child nodes:")
    for cNode in nNode.children:
        nG.remove(cNode)
        pPars = cNode.parents
        #
        #Now create a new child node
        nPars = [N for N in pPars if N not in nodes] + [nNode]
        nCNode = Vertex(cNode.name)
        nCNode.values = [v for v in cNode.values]
        nCNode.parents = nPars
        nG += [nCNode]
        #
        #Now, let's build it's CPT
        #First the nCNode and then it's parents
        probs = []
        #Now, enumerate the values for nodes in nPars
        # Then using nNodeMap, get the correspondig values for 
        #   pPars, then get it's prob
        # We ensure that prob are populated in correct order!
        card = np.prod([n.nCard() for n in nPars])
        nVals = [0]*(len(nPars))
        for nodeValIdx in range(nCNode.nCard()):
            for idx in range(card):
                if idx!= 0 or nodeValIdx != 0:
                    #TODO: Perhaps remove thsi if by setting 
                    #      nVals to all 1
                    for pIdx in range(len(nPars) -1, -1, -1):
                        nVals[pIdx] += 1
                        if nVals[pIdx] < nPars[pIdx].nCard():
                                break
                        nVals[pIdx] = 0
                #
                #Now nVals[0], refers to the new nCnode
                #From this we'll get values for the old parents
                # and use cNode.CPT.getP4mIdx(args) :-)
                # and append that to prob sequentially
                nodeVals = nNodeMap[nVals[-1]]
                """args = [0]*len(cNode.parents)
                for i in range(len(cNode.parents)):
                    n = cNode.parents[i]
                    if n in nCNode.parents:
                        args[i] = nVals[nPars.index(n)]
                    else:
                        #i is refering to nodes
                        # and it has been subsumed by nNode
                        args[i] = nodeVals[nodes.index(n)]
                #More legently as:"""
                args = []
                for n in cNode.parents:
                    if n in nCNode.parents:
                        args += [nVals[nPars.index(n)]]
                    else:
                        #i is refering to nodes
                        # and it has been subsumed by nNode
                        args += [nodeVals[nodes.index(n)]]
                probs += [cNode.CPT.getP4mIdx([nodeValIdx]+args)]
        nCNode.CPT = CPT([nCNode] + nPars, probs)
    setChildren(nG)
    return nG

In [116]:
nG = collapse(G, [B, C], "Z")
print("\n\n",nG[0].name)
A = nG[0]
A.initializePi()

print("\n\n",nG[1].name)
Z = nG[1]
Z.initializePi()

print("\n\n",nG[3].name)
D = nG[3]
D.initializePi()

print("\n\n",nG[2].name)
E = nG[2]
E.initializePi()

[z.name for z in nG]

Now handling child nodes:


 A: Metastatic Cancer


 Z


 D: Coma


 E: Severe headaches


['A: Metastatic Cancer', 'Z', 'E: Severe headaches', 'D: Coma']

In [117]:
order = [A, Z, D, E]
for N in order:
    N.updateBeliefs()

In [118]:
D.nodeLmbda = np.array([1, 0])
E.nodeLmbda = np.array([0, 1])

In [119]:
order = [D, E, Z, A]

In [120]:
for N in order:
    N.updateBeliefs()

In [121]:
A.BEL, Z.BEL, E.BEL, D.BEL

(array([[0.90272374, 0.09727626]]),
 array([[0.88715953, 0.0155642 , 0.08171206, 0.0155642 ]]),
 array([[0., 1.]]),
 array([[1., 0.]]))

In [106]:
D.nodeLmbda, D.nodePI

(array([1, 0]), array([[0.384, 0.616]]))

In [108]:
BEL = np.multiply(
            D.nodeLmbda.reshape(1, -1),
            D.nodePI.reshape(1, -1))

In [111]:
np.sum(BEL)

0.384

In [90]:
Z.nodeLmbda, Z.nodePI, A.inLmbda, Z.outPi

(None,
 array([[0.64, 0.04, 0.28, 0.04]]),
 [array([1., 1.])],
 [array([[0.64, 0.04, 0.28, 0.04]]), array([[0.64, 0.04, 0.28, 0.04]])])

In [91]:
Z.updateBeliefs()
Z.nodeLmbda, Z.nodePI, A.inLmbda, Z.outPi

(array([[1., 1., 1., 1.]]),
 array([[0.64, 0.04, 0.28, 0.04]]),
 [array([[1., 1.]])],
 [array([[0.64],
         [0.04],
         [0.28],
         [0.04]]), array([[0.64],
         [0.04],
         [0.28],
         [0.04]])])

In [54]:
E.nodeLmbda, E.nodePI, Z.inLmbda, E.outPi

(None,
 array([[0.384, 0.616]]),
 [array([1., 1., 1., 1.]), array([1., 1., 1., 1.])],
 [])

In [55]:
E.updateBeliefs()
print("------------")
E.nodeLmbda, E.nodePI, Z.inLmbda, E.outPi

M:  [array([[0.4, 0.6]]), array([[0.2, 0.8]]), array([[0.4, 0.6]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]
------------


(array([1., 1.]),
 array([[0.384, 0.616]]),
 [array([[1., 1., 1., 1.]]), array([1., 1., 1., 1.])],
 [])

In [56]:
D.nodeLmbda, D.nodePI, Z.inLmbda, D.outPi

(None,
 array([[0.68, 0.32]]),
 [array([[1., 1., 1., 1.]]), array([1., 1., 1., 1.])],
 [])

In [57]:
D.updateBeliefs()
print("------------")
D.nodeLmbda, D.nodePI, Z.inLmbda, D.outPi

M:  [array([[0.95, 0.05]]), array([[0.2, 0.8]]), array([[0.2, 0.8]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]
------------


(array([1., 1.]),
 array([[0.68, 0.32]]),
 [array([[1., 1., 1., 1.]]), array([[1., 1., 1., 1.]])],
 [])

In [58]:
order = [D, E, Z, A]

In [59]:
#Now we set values for D and E
D.values
D.nodeLmbda = np.array([1, 0])
E.nodeLmbda = np.array([0, 1])

In [60]:
D.updateBeliefs()

M:  [array([[0.95, 0.05]]), array([[0.2, 0.8]]), array([[0.2, 0.8]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]


In [64]:
Z.inLmbda

[array([[1., 1., 1., 1.]]), array([[0.95, 0.2 , 0.2 , 0.2 ]])]

In [65]:
E.updateBeliefs()

M:  [array([[0.4, 0.6]]), array([[0.2, 0.8]]), array([[0.4, 0.6]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]


In [67]:
Z.inLmbda

[array([[0.6, 0.8, 0.6, 0.8]]), array([[0.95, 0.2 , 0.2 , 0.2 ]])]

In [68]:
Z.nodeLmbda, Z.nodePI

(array([[1., 1., 1., 1.]]), array([[0.64, 0.04, 0.28, 0.04]]))

In [69]:
Z.updateBeliefs()

M:  [array([[0.76, 0.04, 0.19, 0.01]]), array([[0.16, 0.04, 0.64, 0.16]])]
pPiProd after:  A: Metastatic Cancer  is:  [[0.8]
 [0.2]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]


In [70]:
Z.nodeLmbda, Z.nodePI

(array([[0.57, 0.16, 0.12, 0.16]]), array([[0.64, 0.04, 0.28, 0.04]]))

In [71]:
Z.inLmbda

[array([[0.6, 0.8, 0.6, 0.8]]), array([[0.95, 0.2 , 0.2 , 0.2 ]])]

In [81]:
Z.outPi[0]/np.sum(Z.outPi[0]), Z.outPi[1]/np.sum(Z.outPi[1])

(array([[0.89411765],
        [0.01176471],
        [0.08235294],
        [0.01176471]]), array([[0.62337662],
        [0.05194805],
        [0.27272727],
        [0.05194805]]))

In [82]:
A.inLmbda

[array([[0.464, 0.2  ]])]

In [78]:
Z.outPi[0].norm()

AttributeError: 'numpy.ndarray' object has no attribute 'norm'

In [28]:
for n in order:
    n.updateBeliefs()

M:  [array([[0.4, 0.6]]), array([[0.2, 0.8]]), array([[0.4, 0.6]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]
M:  [array([[0.95, 0.05]]), array([[0.2, 0.8]]), array([[0.2, 0.8]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.64]
 [0.04]
 [0.28]
 [0.04]]
M:  [array([[0.76, 0.04, 0.19, 0.01]]), array([[0.16, 0.04, 0.64, 0.16]])]
pPiProd after:  A: Metastatic Cancer  is:  [[0.8]
 [0.2]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]
tLmbda:  [[1.]
 [1.]]


In [40]:
for n in list(reversed(order))[1:]:
    n.updateBeliefs()

M:  [array([[0.76, 0.04, 0.19, 0.01]]), array([[0.16, 0.04, 0.64, 0.16]])]
pPiProd after:  A: Metastatic Cancer  is:  [[0.8]
 [0.2]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]
tLmbda:  [[1.]
 [1.]
 [1.]
 [1.]]
M:  [array([[0.95, 0.05]]), array([[0.2, 0.8]]), array([[0.2, 0.8]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.256]
 [0.008]
 [0.112]
 [0.008]]
M:  [array([[0.4, 0.6]]), array([[0.2, 0.8]]), array([[0.4, 0.6]]), array([[0.2, 0.8]])]
pPiProd after:  Z  is:  [[0.032]
 [0.032]
 [0.224]
 [0.032]]


In [43]:
A.outPi, Z.inLmbda, Z.nodeLmbda

([array([[0.8],
         [0.2]])],
 [array([[0.05, 0.8 , 0.8 , 0.8 ]]), array([[0.4, 0.2, 0.4, 0.2]])],
 array([[0.02, 0.16, 0.32, 0.16]]))

In [39]:
[a.name for a in list(reversed(order))[1:]]

['Z', 'D: Coma', 'E: Severe headaches']

In [33]:
reversed(order)

In [19]:
#Now, belief propagation!!
e = {}
e[3] = False
e[2] = True

for idx in e.keys():
    print(nG[idx].name,': ',e[idx],', val idx: ',
          nG[idx].values.index(e[idx]))

E: Severe headaches :  False , val idx:  0
D: Coma :  True , val idx:  1


In [20]:
"""
To run BP efficiently, one can find topological order and 
follow it.

Or less efficiently, simply iterate and do it. But keep iterating
till any node changes value. A change would  be updated by a 
dirty flag.

However, here we'll do it manually. So no iteration. 
Hard code the seq.
"""
#set all lambdas to 1
for n in nG:
    n.lmbda = [1]*len(n.values)

#Now setting pi for all
A = nG[0]
Z = nG[1]
E = nG[2]
D = nG[3]
A.pi = A.CPT.getCPT()[0]
#Z.pi = 

#Now set BEL as pi, coz lambdas are anyway 1